In [26]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast

In [4]:
def parseData(file):
    for l in open(file,'r'):
        yield json.loads(l)
        
def remove_punctuation(text):
    return ''.join([c.lower() for c in text if c not in set(string.punctuation)])

In [15]:
def get_info(d):
    return [d['review_text'] + ' ' + d['review_summary'], d['fit']]

In [16]:
row_data = list(parseData('renttherunway_final_data.json'))
data = [get_info(d) for d in row_data]

In [17]:
data[1]

['I rented this dress for a photo shoot. The theme was "Hollywood Glam and Big Beautiful Hats". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event.  I felt so glamourous!!!',
 'fit']

In [19]:
def statistic_analyse(data):
    num_fit = 0
    num_small = 0
    num_large = 0
    num_other = 0
    for d in data:
        fit = d[1]
        if (fit == 'fit'): num_fit += 1
        elif (fit == 'small'): num_small += 1
        elif (fit == 'large'): num_large += 1
        else: num_other += 1
    ret = {
        'fit': num_fit,
        'small': num_small,
        'large': num_large,
        'other': num_other
    }
    return ret

In [20]:
statistic_analyse(data)

{'fit': 142058, 'small': 25779, 'large': 24707, 'other': 0}

In [88]:
reviews_fit = []
reviews_small = []
reviews_large = []
cnt = 5
for d in data:
    fit = d[1]
    t = d[0]
    if fit == 'fit' and len(reviews_fit) < cnt:
        reviews_fit.append(t)
    if fit == 'small' and len(reviews_small) < cnt:
        reviews_small.append(t)
    if fit == 'large' and len(reviews_large) < cnt:
        reviews_large.append(t)
    if len(reviews_fit) >= cnt and len(reviews_small) >= cnt and len(reviews_large) >= cnt:
        break

In [89]:
for t in reviews_fit:
    print(t + '\n')

An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments. So many compliments!

I rented this dress for a photo shoot. The theme was "Hollywood Glam and Big Beautiful Hats". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event.  I felt so glamourous!!!

This hugged in all the right places! It was a perfect dress for my event and I received so many compliments on it. Not to mention customer service was great getting this to me in less than 24 hours! It was a great time to celebrate the (almost) completion of my first year of law school.

I rented this for my company's black tie awards banquet.  I liked that this dress was short but was a little fancier with the sequins. (I generally don't care for long dresses.)  I would describe the color as more rose g

In [90]:
for t in reviews_small:
    print(t + '\n')

The dress arrived with a small hole in the beading on the front but wasn't too noticeable. Glad I was able to get two sizes because the 4 was a little tight and would've made for an uncomfortable night of dancing!  It was fun to wear a dress I wouldn't normally buy! 

super cute, 8 was super tight so i wore the 10. if you have booty and hips prob want to get the 10. don't need bra and it has structure to give you shape. the train in the back is extra classy to give it another emphasis for formal.  gala ready 

Lovely fabric, heavier than I imagined it would be (somehow I thought billowy, but its softer and stiffer than that) but it runs small around the ribcage - my normal size gapped 2 inches. I've sent it right back to try the larger size. Pretty! Runs small

Huge on top and super tight in the waist strange fit

Great dress! super comfortable. M fit but too snug for my liking.   Hem hit right above knee. Would rent again ! Love!!



In [91]:
for t in reviews_large:
    print(t + '\n')

This dress was absolutely gorgeous and I received so many compliments!  I wore it to a black tie NYE wedding, and even though it was short, I felt like the fabric, fit, and the way it sparkled made it fit in just fine with the other formal gowns.  I usually wear a 6 or an 8 in dresses.  I ordered a 4 and a 6, and true to other reviewers' comments, the 4 fit me very well.  The chest area was just a tad loose, but that typically happens when I try on dresses...nothing a more padded bra couldn't fix!  The cut and fit were very figure flattering.  By the end of the evening I experience a little chafing under my arms from the sequins, but that was after 5 hours and it was very minor.  Thank you RTR for a great first experience, I'll definitely be back! Stunning dress, perfect for a New Year's Eve wedding!

I ordered this dress as a replacement because I thought my original order would be too small.  This dress fit my abdomen well, but it hung slack around my shoulders and neck.  I wouldn't 

In [42]:
word_count = defaultdict(int)
total_words = 0
punct = string.punctuation
stemmer = PorterStemmer()

In [55]:
for d in data:
    t = d[0]
    t = t.lower()
    t = [c if not (c in punct) else ' ' for c in t] # repalce punct to space
#     t = [c for c in t if not (c in punct)] # non-punct characters
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    for w in words:
        #w = stemmer.stem(w)
        total_words += 1
        word_count[w] += 1

In [56]:
total_words

25327930

In [57]:
len(word_count)

54180

In [58]:
counts = [(word_count[w], w) for w in word_count]
counts.sort()
counts.reverse()

In [59]:
counts[:10]

[(1370533, 'the'),
 (1098484, 'i'),
 (911622, 'a'),
 (842724, 'and'),
 (781726, 'it'),
 (672632, 'dress'),
 (641210, 'was'),
 (501284, 'to'),
 (470007, 'this'),
 (359009, 'for')]

In [72]:
counts[50:60]

[(72068, 'if'),
 (67632, 'bit'),
 (66488, 'really'),
 (65765, 'long'),
 (65118, 'because'),
 (65093, 'definitely'),
 (64656, 'out'),
 (63798, 'flattering'),
 (63655, 'or'),
 (63329, 'length')]

In [67]:
counts[25000:25010]

[(2, 'saflely'),
 (2, 'safisticated'),
 (2, 'safeguards'),
 (2, 'safeand'),
 (2, 'saddled'),
 (2, 'sacks'),
 (2, 'sachi'),
 (2, 'saccharine'),
 (2, 'sabred'),
 (2, 'sa')]

In [68]:
word_count['small']

50835

In [69]:
word_count['large']

33056

In [70]:
word_count['fit']

211022

In [71]:
word_count['perfect']

147467

In [73]:
words = [w[1] for w in counts[:1000]]

In [74]:
words[:10]

['the', 'i', 'a', 'and', 'it', 'dress', 'was', 'to', 'this', 'for']

In [75]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [93]:
wordId['small'], wordId['large'], wordId['tight'], wordId['larger'], wordId['tiny']

(76, 129, 75, 234, 595)

# Train a regressor

In [138]:
def feature(datum):
    feat = [0]*len(wordSet)
    t = datum[0]
    t = t.lower() # lowercase string
    t = [c if not (c in punct) else ' ' for c in t] # repalce punct to space
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    for w in words:
        if not (w in wordSet): continue
        feat[wordId[w]] += 1
    feat.append(1)
    return feat

def lable(datum):
#     ret = [0, 0, 0]
    lables = ['fit', 'small', 'large']
    for i in range(0, len(lables)):
        if datum[1] == lables[i]:
            return i
#             ret[i] = 1
#             break
    return -1

In [94]:
random.seed(1)
random.shuffle(data)
size_tot = len(data)
size_train = int(size_tot * 0.8)
size_dev = int(size_tot * 0.1)
size_test = size_tot - size_train - size_dev
data_train = data[:size_train]
data_dev = data[size_train:size_train + size_dev]
data_test = data[-size_test:]

In [95]:
len(data_train), len(data_dev), len(data_test)

(154035, 19254, 19255)

In [105]:
from sklearn.linear_model import SGDClassifier

In [97]:
from sklearn.linear_model import LogisticRegression

In [106]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

In [116]:
X = [feature(d) for d in data_train]
y = [lable(d) for d in data_train]

In [117]:
from sklearn import svm

In [122]:
lin_clf = svm.LinearSVC()
lin_clf.fit(X, y) 

/Users/czf/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [143]:
X_dev = [feature(d) for d in data_dev]
y_dev = [lable(d) for d in data_dev]
dec = lin_clf.decision_function(X_dev)

In [144]:
y_dev[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]

In [150]:
def dev2res(datum):
    ret = 0
    for i in range(1, len(datum)):
        if datum[i] > datum[ret]:
            ret = i
    return ret
        
def evaluate(y, pred):
    pred = [dev2res(d) for d in pred]
    t_fit, f_fit, t_small, f_small, t_large, f_large = 0, 0, 0, 0, 0, 0
    correct = 0
    tot, tot_fit, tot_small, tot_large = len(y), 0, 0, 0
    
    for i in range(0, tot):
        if pred[i] == y[i]:
            correct += 1
        if pred[i] == 0: #fit
            if y[i] == pred[i]: t_fit += 1
            else: f_fit += 1
            tot_fit += 1
        elif pred[i] == 1: #small
            if y[i] == pred[i]: t_small += 1
            else: f_small += 1
            tot_small += 1
        else:
            if y[i] == pred[i]: t_large += 1
            else: f_large += 1
            tot_large += 1
    
    precision_fit = t_fit / (t_fit + f_fit)
    precision_small = t_small / (t_small + f_small)
    precision_lager = t_large / (t_large + f_large)
    
    recall_fit = t_fit / tot_fit
    recall_small = t_small / tot_small
    recall_large = t_large / tot_large
    
    acc = correct / tot
    
    return {
        'acc': acc,
        'precision_fit': precision_fit, 
        'precision_small': precision_small, 
        'precision_lager': precision_lager, 
        'recall_fit': recall_fit, 
        'recall_small': recall_small, 
        'recall_large': recall_large, 
    }
    

In [151]:
evaluate(y_dev, dec)

{'acc': 0.8051314012672691,
 'precision_fit': 0.816711915535445,
 'precision_small': 0.7111913357400722,
 'precision_lager': 0.7573415765069552,
 'recall_fit': 0.816711915535445,
 'recall_small': 0.7111913357400722,
 'recall_large': 0.7573415765069552}

In [153]:
theta = lin_clf.coef_

In [159]:
for i in range(0, 3):
    weight = [(w, theta[i][wordId[w]]) for w in wordSet]
    weight.sort(key=lambda tup: tup[1])
    weight.reverse()
    for j in range(0, 50):
        print(weight[j])
    print()

('true', 0.7571825051953963)
('57', 0.6856793031700595)
('ive', 0.24567619356937026)
('deal', 0.24329006282118168)
('youre', 0.19280740545364602)
('chested', 0.1915635482142574)
('ride', 0.1697952998850147)
('between', 0.1601586928906236)
('depending', 0.15014589464131461)
('dream', 0.14057191092226481)
('nervous', 0.13986959126689294)
('fix', 0.13683101830387187)
('cup', 0.12643917109273203)
('glove', 0.12564038465434743)
('slimming', 0.12004221800110676)
('excellent', 0.11646162336380528)
('thats', 0.11505472804304309)
('perfectly', 0.1127677509670407)
('bought', 0.10959951525118483)
('buying', 0.10657660776852672)
('weeks', 0.10237673364169757)
('bother', 0.10166363852043488)
('worried', 0.09838285206517286)
('trouble', 0.09680463912893308)
('pear', 0.09522761374396335)
('princess', 0.09393512900410303)
('mesh', 0.09316273113319538)
('team', 0.09191228531440375)
('sat', 0.09127825633165786)
('concerned', 0.0902430250043143)
('wouldnt', 0.09007081956010972)
('noticed', 0.088362957837

In [162]:
statistic_analyse(data_dev)

{'fit': 14105, 'small': 2614, 'large': 2535, 'other': 0}

In [163]:
statistic_analyse(data_dev)

{'fit': 14105, 'small': 2614, 'large': 2535, 'other': 0}

In [164]:
statistic_analyse(data_train)

{'fit': 113704, 'small': 20574, 'large': 19757, 'other': 0}